In [22]:
import torch
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForMaskedLM, Trainer, TrainingArguments


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [24]:
dataset = load_dataset("code_search_net", "python")

In [25]:
dataset.column_names

{'train': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'test': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'validation': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url']}

In [26]:
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base").to(device)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
def tokenize_code(code_snippet, tokenizer):
    return tokenizer(
        code_snippet, max_length=512, truncation=True, padding='max_length', return_tensors="pt"
    )

In [28]:
def preprocess_data(batch, tokenizer):
    batch["input_ids"] = tokenize_code(batch["func_code_string"], tokenizer)["input_ids"]
    batch["labels"] = tokenize_code(batch["func_documentation_string"], tokenizer)["input_ids"]
    return batch

In [29]:
train_dataset = dataset["train"].shuffle(seed=42).select(range(1000)).map(lambda x: preprocess_data(x, tokenizer), batched=True)
eval_dataset = dataset["validation"].shuffle(seed=42).select(range(200)).map(lambda x: preprocess_data(x, tokenizer), batched=True)

In [30]:
training_args = TrainingArguments(
    output_dir="./codebert-finetuned",    # output directory
    num_train_epochs=3,                   # number of epochs
    per_device_train_batch_size=16,       # batch size
    per_device_eval_batch_size=16,        # eval batch size
    warmup_steps=500,                     # warmup steps
    weight_decay=0.01,                    # weight decay
    logging_dir="./logs",                 # logging directory
    logging_steps=10,
    evaluation_strategy="steps",          # Evaluation after every logging step
    save_total_limit=2,                   # Keep only last two checkpoints
    save_steps=500,                       # Save model every 500 steps
    report_to="none",
    fp16=True                     # No reports (e.g., to wandb)
)

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:463: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

  0%|          | 0/189 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  5%|▌         | 10/189 [04:44<1:24:23, 28.29s/it]

{'loss': 18.0067, 'grad_norm': 62.93367385864258, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.16}
